<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>


# Non-Negative Matrix Factorization

Estimated time needed: **45** minutes





You've been hired by a video streaming platform that specializes in kids content. The company would like to ensure that all content on the platform does not infringe on a copyright written material. The company has created an image database of copyrighted material, not only must you detect the same images you must detect similar images. Your  job is to come up with an image retrieval system to find the most similar image to each image in the provided dataset. Because your model is used for legal purposes it must be interpretable, this means it should be able to determine why the images are similar. As a result, you will use Non-Negative Matrix factorization since the factorized matrices can be interpreted as real images. You will use the [Anime Face Dataset](https://www.kaggle.com/datasets/splcher/animefacedataset?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) from kaggle.



<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/fake_cartoon.png" alt="cognitiveclass.ai logo">

<a href="https://www.piratesandprincesses.net/the-story-of-goodtimes-and-their-numerous-disney-knockoffs/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01"> image source</a>


## Table of Contents

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li><a href="#Datasets">Datasets</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li>
        <a href="#Background">Background</a>
        <ol>
               <li><a href="#What's a Non-Negative Matrix Fatorization?">What's a Non-Negative Matrix Fatorization?</a></li>
        </ol>
    </li>
  <li><a href="#Applying Non-Negative Matrix Factorization">Applying Non-Negative Matrix Factorization</a></li>
  <li><a href="#Image Retrieval System">Image Retrieval System</a></li>
   <li><a href="#Exercises">Exercises</a>
       <ol>
            <li><a href="#Exercise 1">Exercise 1</a></li>
            <li><a href="#Exercise 2">Exercise 2</a></li>
            <li><a href="#Exercise 3">Exercise 3</a></li>
            <li><a href="#Exercise 4">Exercise 4</a></li>
            <li><a href="#Exercise 5">Exercise 5</a></li>
        </ol>
    </li> 
 </ol>    
     
    
     

## Objectives

After completing this lab you will be able to:

- __Understand__ Basics Matrix operations  such as Matrix addition  , Vector Multiplication，and Eigen decomposition

- __Apply__ Apply these Matrix operations using numpy


## Datasets

Download and unzip the **images** dataset: 


In [1]:
import skillsnetwork

await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/images.tar.gz")

  0%|          | 0/20006 [00:00<?, ?it/s]

Saved to '.'


In [3]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [5]:
from numpy.random import RandomState
from sklearn.decomposition import NMF
from os import listdir, getcwd
from os.path import isfile, join
from PIL import Image, ImageOps

In [4]:
import warnings
warnings.simplefilter('ignore')

### Defining Helper Functions


Load train and test set:


In [6]:
def get_data_matrix(test=False, Length=100, Width=100, mypath="images/"):

    files = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f)) and f[0] != '.']
    if mypath + '/.DS_Store' in files:
        files.remove(mypath + '/.DS_Store')
  
    if test:
        print("test data")
        files = files[9000: 10000]
        
    else:
        print("training data")
        files = files[0: 9000]
        
    print(len(files))
    X = np.zeros((len(files), Length * Width))

    for i, file in enumerate(files):
        img = Image.open(file).resize((Width, Length))
        img = ImageOps.grayscale(img)
        I = np.array(img)
 
        X[i, :] = I.reshape(1, -1)
    return X

Plot image vectors:


In [7]:
def reshape_row(x):
    plt.imshow(x.reshape(Length, Width), cmap="gray")

This threshold function outputs an index if the input ```similar_distance``` is grater than or less than ```min_``` and ```max_```.


In [8]:
def threshold(similar_distance, max_=0.1, min_=0):
    dataset_index = np.where(np.logical_and(similar_distance > min_, similar_distance < max_))[0]
    query_index = similar_index[np.logical_and(similar_distance > min_, similar_distance < max_)]
    return dataset_index, query_index

Plot dataset images and query images, ```X``` and ```X_q``` are global variables.


In [9]:
def plot_data_query(dataset_index, query_index, N):
    
    for data_sample, query_sample in zip(dataset_index[0: N], query_index[0: N]):
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        reshape_row(X[data_sample])
        plt.title("dataset sample {}".format(data_sample))
        plt.subplot(1, 2, 2)
        reshape_row(X_q[query_sample])
        plt.title("query sample match {}".format(query_sample))
        plt.show()
        print("-----------------------------------------------------")